# Segmented Spacetime — Colab Auto Runner + PPN/Consistency Tests

One‑click run for SSZ auto‑evaluation **and** built‑in checks:
- Core metrics (median |Δz|, paired sign test, equal‑count mass bins)
- PPN far‑field checks (beta=gamma=1), energy conditions, C¹/C² continuity proxies
- Optional: shadow/QNM eikonal sanity (toggle)

**No repo clone required** — the notebook fetches just the needed raw files.

In [ ]:
#@title Setup (minimal deps)import sys, subprocesssubprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'numpy', 'pandas', 'matplotlib'])print('OK — deps ready')

In [ ]:
#@title Fetch required files (raw; no git clone)import urllib.request, hashlib, osfrom pathlib import PathBASE = 'https://raw.githubusercontent.com/LinoCasu/Segmented-Spacetime-Mass-Projection-Unified-Results/main'FILES = ['real_data_full.csv', 'segspace_all_in_one_extended.py', 'test_ppn_exact.py', 'test_energy_conditions.py', 'test_c1_segments.py', 'test_c2_segments_strict.py', 'test_c2_curvature_proxy.py', 'shadow_predictions.py', 'qnm_eikonal.py', 'eht_bound_template.py']def dl(p):    url = f"{BASE}/{p}"    out = Path(p)    out.parent.mkdir(parents=True, exist_ok=True)    with urllib.request.urlopen(url) as r, open(out, 'wb') as f:        f.write(r.read())    return outdef sha256(p):    import hashlib    h = hashlib.sha256()    with open(p, 'rb') as f:        for chunk in iter(lambda: f.read(8192), b''):            h.update(chunk)    return h.hexdigest()for f in FILES:    p = dl(f)    print('OK:', p, sha256(p))

In [ ]:
#@title Define core SSZ auto‑runner (equal‑count bins, optional artifacts)import os, json, math, time, hashlib, textwrapfrom pathlib import Pathimport numpy as np, pandas as pd, matplotlib.pyplot as pltM_sun = 1.98847e30BANNER = '='*88; SUB='-'*88def fmt_sig(x, sig=3):    try: x=float(x)    except: return 'n/a'    if not np.isfinite(x): return 'n/a'    ax=abs(x)    if (ax!=0 and (ax>=1e6 or ax<1e-3)): return f"{x:.{sig}e}"    s=f"{x:.{sig}g}";    if 'e' not in s and '.' not in s: s+= '.0'    return sdef sha256(p: Path):    h=hashlib.sha256()    with open(p,'rb') as f:        for ch in iter(lambda: f.read(8192), b''): h.update(ch)    return h.hexdigest()def finite(x):    try: return np.isfinite(float(x))    except: return Falsedef maybe_save_raws(raws, outdir, enable=False):    if not enable: return None    outdir.mkdir(parents=True, exist_ok=True)    p = outdir / 'raws_full.csv'    pd.DataFrame(raws).to_csv(p, index=False)    return pdef maybe_plots(raws, outdir, enable=False):    if not enable: return []    outdir.mkdir(parents=True, exist_ok=True)    df = pd.DataFrame(raws); figs=[]    for key,label in [('abs_seg','SEG'),('abs_grsr','GR×SR'),('abs_gr','GR'),('abs_sr','SR')]:        if key in df and df[key].notna().any():            plt.figure(figsize=(7,5))            df[key].dropna().astype(float).plot(kind='hist', bins=30)            plt.xlabel('|Δz|'); plt.ylabel('Count'); plt.title(f'Histogram of |Δz| — {label}')            p = outdir / f'hist_abs_{label.replace("×","x")}.png'            plt.tight_layout(); plt.savefig(p, dpi=300); plt.close(); figs.append(p)    if {'abs_seg','abs_grsr'}.issubset(df.columns):        s = df['abs_seg'].dropna().astype(float).sort_values(); g = df['abs_grsr'].dropna().astype(float).sort_values()        if len(s)>0 and len(g)>0:            ys=(np.arange(1,len(s)+1))/len(s); yg=(np.arange(1,len(g)+1))/len(g)            plt.figure(figsize=(7,5))            plt.plot(s.values, ys, label='SEG'); plt.plot(g.values, yg, label='GR×SR')            plt.xlabel('|Δz|'); plt.ylabel('ECDF'); plt.title('ECDF — SEG vs GR×SR'); plt.legend()            p = outdir / 'ecdf_abs_SEG_vs_GRSR.png'            plt.tight_layout(); plt.savefig(p, dpi=300); plt.close(); figs.append(p)    if {'abs_seg','abs_grsr'}.issubset(df.columns):        s=df['abs_seg'].dropna().astype(float).values; g=df['abs_grsr'].dropna().astype(float).values        if len(s)>0 and len(g)>0:            plt.figure(figsize=(7,5))            plt.boxplot([s,g], labels=['SEG','GR×SR'])            plt.ylabel('|Δz|'); plt.title('SEG vs GR×SR — |Δz|')            p = outdir / 'box_abs_SEG_vs_GRSR.png'            plt.tight_layout(); plt.savefig(p, dpi=300); plt.close(); figs.append(p)    return figsdef run_core(save_raws=False, plots=False):    import runpy    csv_path = Path('real_data_full.csv'); mod_path = Path('segspace_all_in_one_extended.py')    ns = runpy.run_path(str(mod_path))    load_csv = ns['load_csv']; evaluate_redshift = ns['evaluate_redshift']    z_gravitational = ns['z_gravitational']; z_special_rel = ns['z_special_rel']    z_combined = ns['z_combined']; z_seg_pred = ns['z_seg_pred']    A=float(ns['A']); B=float(ns['B']); ALPHA=float(ns['ALPHA'])    print(BANNER); print(' SEGMENTED SPACETIME — AUTO RUN (NO ARGS)'); print(BANNER)    print('Deterministic SSZ evaluation with phi/2 coupling and fixed Delta(M).\n')    print(BANNER); print(' INPUTS & PROVENANCE (REPRODUCIBILITY)'); print(BANNER)    print('CSV file     :', csv_path.resolve()); print('CSV sha256   :', sha256(csv_path))    print('Module file  :', mod_path.resolve()); print('Module sha256:', sha256(mod_path))    print('\n'+BANNER); print(' PHYSICAL CONSTANTS & MODEL CHOICES'); print(BANNER)    print(f'Delta(M) parameters (fixed): A={A}, B={B}, ALPHA={ALPHA} (slow; dimensionless after norm).')    print('PPN outer series: beta=gamma=1 (unchanged).')    rows = load_csv(csv_path)    print(f'[ECHO] [OK] loaded rows: {len(rows)}')    res = evaluate_redshift(rows=rows, prefer_z=True, mode='hybrid', dmA=A, dmB=B, dmAlpha=ALPHA,                            lo=None, hi=None, drop_na=False, paired_stats=True, n_boot=0, bins=8,                            do_plots=False, out_fig_dir=None, filter_complete_gr=True)    med = res.get('med', {}); paired = res.get('paired', {})    # Residuals    raws=[]    for r in rows:        z_obs=r.get('z'); z_geom=r.get('z_geom_hint')        z_use=float(z_obs) if finite(z_obs) else (float(z_geom) if finite(z_geom) else None)        Msun=float(r.get('M_solar')) if finite(r.get('M_solar')) else float('nan')        Mkg=Msun*M_sun if finite(Msun) else float('nan')        rem=float(r.get('r_emit_m')) if finite(r.get('r_emit_m')) else float('nan')        v_los=float(r.get('v_los_mps',0.0)) if finite(r.get('v_los_mps',0.0)) else 0.0        v_tot=float(r.get('v_tot_mps',float('nan'))) if finite(r.get('v_tot_mps',float('nan'))) else float('nan')        if not (finite(Mkg) and finite(rem) and rem>0 and z_use is not None): continue        zgr=z_gravitational(Mkg, rem); zsr=z_special_rel(v_tot, v_los); zgrsr=z_combined(zgr, zsr)        import math        lM = math.log10(Mkg) if finite(Mkg) and Mkg>0 else math.log10(M_sun)        zseg = z_seg_pred('hybrid', float(z_geom) if finite(z_geom) else None, zgr, zsr, zgrsr, A, B, ALPHA, lM, None, None)        def adiff(zm):            try: return abs(float(z_use)-float(zm))            except: return float('nan')        raws.append({'case': r.get('case'), 'M_solar': Msun, 'abs_seg': adiff(zseg), 'abs_grsr': adiff(zgrsr), 'abs_gr': adiff(zgr), 'abs_sr': adiff(zsr)})    # Equal-count bins    valid=[row for row in raws if finite(row['M_solar'])]; valid.sort(key=lambda r: r['M_solar'])    bins=[]    if len(valid)>=8:        parts=np.array_split(np.arange(len(valid)), 8)        for i, idxs in enumerate(parts):            sel=[valid[j] for j in idxs]            lo=sel[0]['M_solar']; hi=sel[-1]['M_solar']            def med_key(k):                s=[x[k] for x in sel if finite(x[k])]                return float(pd.Series(s).median()) if len(s)>0 else float('nan')            bins.append({'bin':i,'lo':float(lo),'hi':float(hi),'count':len(sel),'med_seg':med_key('abs_seg'),'med_grsr':med_key('abs_grsr')})    else:        def med_key_all(k):            s=[x[k] for x in valid if finite(x[k])]            return float(pd.Series(s).median()) if len(s)>0 else float('nan')        bins=[{'bin':0,'lo':float('nan'),'hi':float('nan'),'count':len(valid),'med_seg':med_key_all('abs_seg'),'med_grsr':med_key_all('abs_grsr')}]    # Print    print(BANNER); print(' ACCURACY — MEDIAN ABS ERROR |Δz|'); print(BANNER)    for k,n in [('seg','SSZ (phi/2 + DeltaM)'),('grsr','GR×SR'),('gr','GR'),('sr','SR')]:        if k in med and med[k] is not None: print(f"{n:<22}: {med[k]:.6g}")    if paired:        print('\n'+SUB); print(' PAIRED SIGN-TEST — SSZ vs GR×SR'); print(SUB)        for k in ['N_pairs','N_Seg_better','share_Seg_better','binom_two_sided_p']: print(f"{k:<22}: {paired.get(k)}")    print('\n'+SUB); print(' MASS-BINNED MEDIANS (SEG vs GR×SR) — with counts'); print(SUB)    print(f"{'bin':>3} | {'lo→hi (M☉)':<24} | {'count':>5} | {'med_seg':>12} | {'med_grsr':>12}")    print('-'*70)    for b in bins:        rng=f"[{fmt_sig(b['lo'])}…{fmt_sig(b['hi'])}]"        print(f"{b['bin']:>3} | {rng:<24} | {b['count']:>5} | {float(b['med_seg']):>12.6g} | {float(b['med_grsr']):>12.6g}")    # Save    repdir=Path('full_pipeline/reports'); figdir=Path('full_pipeline/figures'); repdir.mkdir(parents=True, exist_ok=True)    summary={'med':med,'paired':paired,'bins':bins,'raws':raws,'provenance':{'csv':{'path':str(csv_path.resolve()),'sha256':sha256(csv_path)},'module':{'path':str(mod_path.resolve()),'sha256':sha256(mod_path)},'runner':'colab-v4-ppn'}}    out = repdir/'summary_full_terminal_v4_ppn.json'    with open(out,'w',encoding='utf-8') as f: json.dump(summary,f,indent=2)    raw_csv=maybe_save_raws(raws, repdir, enable=save_raws); figs=maybe_plots(raws, figdir, enable=plots)    print(BANNER); print(' RUN COMPLETE'); print(BANNER)    print('Summary JSON             :', out.resolve())    if raw_csv: print('Raw residuals CSV        :', raw_csv.resolve())    if figs:        print('Figures:');        for p in figs: print(' -', p.resolve())

In [ ]:
#@title PPN & consistency test runner (toggle heavy tests)RUN_SHADOW = False  #@param {type:"boolean"}RUN_QNM    = False  #@param {type:"boolean"}import runpy, tracebackdef run_script(path):    print('='*60); print('RUN', path); print('='*60)    try:        runpy.run_path(path, run_name='__main__')        print('[OK] completed:', path)    except SystemExit as e:        print('[EXIT]', path, e)    except Exception as e:        print('[ERROR]', path, e); traceback.print_exc()for p in ['test_ppn_exact.py','test_energy_conditions.py','test_c1_segments.py','test_c2_segments_strict.py','test_c2_curvature_proxy.py']:    run_script(p)if RUN_SHADOW:    run_script('shadow_predictions.py')if RUN_QNM:    run_script('qnm_eikonal.py')print('PPN/consistency suite done.')

In [ ]:
#@title Execute core run (toggle artifacts)SAVE_RAWS = True  #@param {type:"boolean"}PLOTS = True      #@param {type:"boolean"}run_core(save_raws=SAVE_RAWS, plots=PLOTS)